In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)

dataset = datasets[0]

df_enhanced.csv


In [11]:
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold


# 1. VarianceThreshold
# Определим порог. Признаки с дисперсией ниже этого порога будут удалены.
# Начнем с очень низкого порога, чтобы отфильтровать только совсем уж бесполезные признаки.
# Для бинарных признаков (Wilderness_Area, Soil_Type), низкая дисперсия может означать,
# что подавляющее большинство значений равно 0 или 1, что может быть информативным.
# Поэтому применим VarianceThreshold только к числовым признакам, а затем к бинарным.
# Или, лучше, применим его ко всему X, но с порогом, который учитывает размах значений.

# Для начала, исключим бинарные признаки из рассмотрения VarianceThreshold, если они сильно различаются.
# Но, учитывая, что они уже One-Hot encoded, и они могут быть разряженными,
# VarianceThreshold может быть полезен для их фильтрации, если класс представлен слишком редко.
# Однако, лучше использовать VarianceThreshold после One-Hot Encoding, если бы он не был сделан.
# В данном случае, бинарные признаки уже в X_original.
X_original = dataset.drop(dataset.target_features, axis=1)

# Порог, например, 0.01.
selector = VarianceThreshold(threshold=0.01) # Можно настроить порог
selector.fit(X_original)
# Получаем названия признаков, которые прошли отбор
features_after_variance_threshold = X_original.columns[selector.get_support()]
print(features_after_variance_threshold)
print(len(features_after_variance_threshold), dataset.shape)

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_0',
       'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3',
       'Soil_Type_3', 'Soil_Type_5', 'Soil_Type_9', 'Soil_Type_10',
       'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_19', 'Soil_Type_21',
       'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_28', 'Soil_Type_29',
       'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_37',
       'Soil_Type_38', 'Soil_Type_39', 'Distance_to_Water', 'Avg_Hillshade',
       'Hydro_Road_Fire_Distance', 'Elevation_x_Slope'],
      dtype='object')
36 (505354, 59)


In [17]:

print(f"\nКоличество признаков после VarianceThreshold (порог 0.01): {len(features_after_variance_threshold)}")
print(f"Удаленные признаки VarianceThreshold: {list(set(X_original.columns) - set(features_after_variance_threshold))}")

# Создадим DataFrame с признаками после VarianceThreshold
X_vt = X_original[features_after_variance_threshold]

# 2. SelectKBest с f_classif
# Выберем, например, 30 лучших признаков. Это число можно подбирать.
# Поскольку у нас 54 признака, 30 - это значительное сокращение.
k_best_selector = SelectKBest(score_func=f_classif, k=30) # Можно попробовать и другие значения K
k_best_selector.fit(X_vt, dataset[dataset.target_features]) # Используем данные после VarianceThreshold
features_selected_kbest = X_vt.columns[k_best_selector.get_support()]

print(f"\nКоличество признаков после SelectKBest (k=30): {len(features_selected_kbest)}")
print(f"Отобранные признаки SelectKBest: {list(features_selected_kbest)}")

# Сформируем окончательный набор данных из существенных признаков
df_selected_features = dataset[features_selected_kbest.tolist() + dataset.target_features]


print(f"Размер сформированного набора данных: {df_selected_features.shape}")


Количество признаков после VarianceThreshold (порог 0.01): 36
Удаленные признаки VarianceThreshold: ['Soil_Type_7', 'Soil_Type_34', 'Soil_Type_0', 'Soil_Type_25', 'Soil_Type_6', 'Soil_Type_4', 'Soil_Type_16', 'Soil_Type_2', 'Soil_Type_13', 'Soil_Type_8', 'Soil_Type_36', 'Soil_Type_17', 'Soil_Type_1', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_27', 'Soil_Type_35', 'Soil_Type_26', 'Soil_Type_20', 'Soil_Type_18', 'Soil_Type_24', 'Soil_Type_33']

Количество признаков после SelectKBest (k=30): 30
Отобранные признаки SelectKBest: ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_0', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Soil_Type_3', 'Soil_Type_5', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_31', 'Soil_Type_37',

S:\anaconda\envs\study\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
df_selected_features.to_csv("../datasets/df_selected_features.csv", index=False)
